In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Visualizing Large  / High Dimensional Datasets

## Unique Concerns with Large Data

When attempting to visualize large or high-dimensional datasets, our concerns are twofold:

1. (you-facing) Effectively computing the visualization in code.
2. (end-user-facing) Effectively conveying the key points to viewers.

Both of these concerns stem from the same issue: the dataset simply contains tons of information, and it's your job to separate the signal from the noise both for reasons of computational tractability and visualization efficacy.

There are a couple ways ways to reduce the amount of data your visualization relies on. We'll discuss

- preprocessing our dataset (eg subsampling, PCA)
- filtering or querying in realtime for subsets/groups of the data (assuming you have a powerful enough backend)

## PCA And Dimensionality Reduction

As a technique for dimensionality reduction, [PCA](https://en.wikipedia.org/wiki/Principal_component_analysis) is very useful (see Module 3 for more info on PCA). One way we can use this to our advantage is by mapping high-dimensional data into 2D space for quick and easy visualizations. 

In [ ]:
# https://github.com/glemaitre/UnbalancedDataset/blob/master/notebook/Notebook_UnbalancedDataset.ipynb
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification


# Generate some data
X, y = make_classification(n_classes=2, class_sep=2, weights=[0.1, 0.9],
                           n_informative=3, n_redundant=1, flip_y=0,
                           n_features=20, n_clusters_per_class=1,
                           n_samples=1000, random_state=10)

# Instanciate a PCA object for the sake of easy visualisation
pca = PCA(n_components = 2)

# Fit and transform x to visualise inside a 2D feature space
x_vis = pca.fit_transform(X)

# Plot the original data
# Plot the two classes
palette = sns.color_palette()
plt.scatter(x_vis[y==0, 0], x_vis[y==0, 1], label="Class #0", alpha=0.5, 
            edgecolor='#262626', facecolor=palette[0], linewidth=0.15)
plt.scatter(x_vis[y==1, 0], x_vis[y==1, 1], label="Class #1", alpha=0.5, 
            edgecolor='#262626', facecolor=palette[2], linewidth=0.15)

plt.legend()
plt.show()

This is a fairly contrived example. In general, further explanation is likely needed if you decide to use PCA or other dimensionality-reduction techniques to visualize some subset or aggregate of your data. For a more interesting application, check out this visualization of word2vec feature space clustering ([github](http://github.com/ronxin/wevi)) ([online demo](http://bit.ly/wevi-online)) (from [this paper](http://www-personal.umich.edu/~ronxin/pdf/w2vexp.pdf)).

Two other dimensionality reduction alternatives: [MDS](https://en.wikipedia.org/wiki/Multidimensional_scaling) and [Denoising Autoencoders](https://en.wikipedia.org/wiki/Autoencoder#Denoising_Autoencoder).

## Using Pandas to filter and downsample for visualization

### High-level considerations

When visualizing data, it usually makes litte sense to try to convey gigabytes' (or more) worth of information to an end-user (unless your end-user is [Adrian Veidt](http://i46.tinypic.com/1qqtmw.jpg)). Plus, this slows down computation on the backend. A little pre-processing goes a long way toward addressing both issues 1) and 2) from the beginning of this notebook.

Some considerations:
- When downsampling timeseries data, make sure that your sample is sufficiently representative to include time-based effects. Can you think of some examples?
- Consider visualizing a deliberate subset (not via sampling) of the data. Using some subset, can you successfully reinforce the message you're trying to convey? Do you want to provide another visualization to compare/contrast with the rest of the data?


### Tips and tricks from the field in pandas

In [ ]:
import pandas as pd

names =["State_Code", "County_Code", "Census_Tract_Number", "NUM_ALL", "NUM_FHA", "PCT_NUM_FHA", "AMT_ALL", "AMT_FHA", "PCT_AMT_FHA"]
df = pd.read_csv('small_data/fha_by_tract.csv', names=names)  ## Loading a CSV file, without a header (so we have to provide field names)

"""
Borrowed from the Pandas notebook (see module 1 for more info).
- Notice how the distribution of the two histograms is the same. 
- If you don't care about raw counts, sampling speeds up visualization
while conveying the exact same information.
"""
df['PCT_AMT_FHA'].hist(bins=50, alpha=0.5)
df['PCT_AMT_FHA'].sample(frac=.5).hist(bins=50, alpha=0.5)

In the visualization miniproject, we provide a pre-cleaned ~80MB dataset. We did a fair bit of processing from a week's worth of raw CSV bus time data (about 300MB each). If you'd like to try your hand at downsampling yourself, we've provided some of that code here, highlighting the techniques we used to make the data Heroku-sized.

In [ ]:
from datetime import datetime

# the raw MTA data actually needs a slightly different parser, if you copy this for your own uses
date_parser=lambda u: pd.datetime.strptime(u, "%Y-%m-%d %H:%M:%S")
CSV_FILENAMES=["small_data/bustime_micro.csv"]
archive = pd.concat([pd.read_csv(fname,  
                                 parse_dates=True,
                                 date_parser=date_parser,
                                 index_col=0) for fname in CSV_FILENAMES])
#query and clean
live_archive = archive[archive.block_assigned > 0] # "assigned" to a route

#throw away trips with <15 reports, probably not real
good_trips_only = live_archive.groupby(live_archive.trip_id).filter(lambda group: len(group) > 15)

In [ ]:
## join to trip data and filter to manhattan only (ie look at a subset)
## Note the "reset_index" to deal with merging index complications.
# good_trips_only.reset_index(inplace=True)
# merged = good_trips_only.merge('trips.txt', on='trip_id', left_index=True)
# merged.sort('timestamp', inplace=True)
# merged.set_index('timestamp')
# manhattan_only = merged[merged.route_id.str.contains("^M")]

In [ ]:
def get_intervals_for_archive():
    intervals = [(datetime(2015, 9, d, h, m),
                   datetime(2015, 9, d, h, m + 2))
                for h in range(0, 24)
                for m in range(0, 60, 15)
                for d in range(13, 21)]
    return intervals

#exercise: any better solution for this?
subsampled_dataset = pd.concat([good_trips_only[lower:upper]
                                for lower, upper in get_intervals_for_archive()])

We can still load 7 days' worth of data into memory and preprocess without too much trouble. What we have a dataset that won't fit into RAM?

Pandas has a solution for that: `skiprows`

In [ ]:
import pandas as pd
 
f = "small_data/fha_by_tract.csv"

with open(f) as in_file:
    num_lines = sum(1 for l in in_file)

n = 10
# make sure line 0 is not included if you have a header!
skip_list = [x for x in range(1, num_lines) if x % n != 0]

# Read the data
%time data1 = pd.read_csv(f, skiprows=skip_list)

In [ ]:
# a generator will also work. note it is slower.
skip_gen = (x for x in range(1, num_lines) if x % n != 0)

%time data2 = pd.read_csv(f, skiprows=skip_gen)

## On the web - live-query a backend

One of the more powerful aspects of web-based/interactive visualizations is the ability to query and render data in realtime. This allows us to, in some contexts, live-query for some subset of the data and effectively downsample this way.

- On the web, we always have to think about server -> client bandwidth considerations. 
- Remember, end-users don't necessarily have modern, powerful computers.
- Always assume the worst (the visualization equivalent of the UI/UX adage "assume the user is drunk")


## Do it in Python: the Bokeh Server
One of Bokeh's most powerful abstractions is the Bokeh Server, which lets you run a visualization server in the background that can live-query, process, filter, and compute visualizations using your more powerful backend infrastructure. For example, you might wish to let a user live-query site log data, or hit information, over varying timespans.

Check out the [Bokeh Gallery](http://bokeh.pydata.org/en/latest/docs/gallery.html#gallery) for some inspiration, and the [Bokeh Docs](http://bokeh.pydata.org/en/0.11.0/docs/user_guide/server.html) explain how the update functionality/synchronization works.

Under the hood, Bokeh is doing the same AJAX magic as the D3 example below, but you don't have to concern yourself as much with the details.

If you want to play around with the Bokeh server on Heroku, it does work! [With some fiddling](https://github.com/birdsarah/bokeh-server-heroku).

## Do it on the Web: D3.js and AJAX calls.

AJAX (advanced Javascript and XML) is a powerful web paradigm that allows for asynchronous calls between a backend server and a frontend browser. 

![Here, have a fancy diagram](http://www.openajax.org/images/AjaxContinuousExperience.gif)


For example, if you're running code in Jupyter notebooks, you're using AJAX to 

1. send your code to the backend Jupyter server.
2. receive the output after the Jupyter server executes your code.

In the context of visualization, this allows us to live-query some subset of view of the data, process appropriately on the backend, and simply render on the frontend. Here's an example for the bus heatmap miniproject:

### JS (frontend)
```javascript
//NOTE: requires the underscore.js library
//NOTE: this code will slam the backend with requests because it has no delay.
// It's simply meant to showcase some asynchronous code that loops over small intervals.
// We could use the setTimeout function in javascript to delay this code.

_.range(24).foreach(function(hours){
    _.range(0, 60, 15).foreach(function(minutes){
        // use AJAX to query bus positions at 
        var swipes = d3.csv("/positions_at_time.csv?hours=" + hours + '&minutes=' + minutes)
        .get(function(error, rows){
          var points = rows.map(function(r){
            return  [
              +r.latitude,
              +r.longitude,
            ];
        });
    });
});
```

### Python (backend)
```python
# NOTE: code here written for Flask framework
@app.route('/positions_at_time.csv')
def positions_at_time():
  hours = int(request.args.get('hours', 12))
  minutes = int(request.args.get('minutes', 30))
  
  #could easily make this parameter modifiable
  date_for_data = datetime.date(2015, 9, 17)
  
  time_for_data = datetime.time(hours, minutes)
  
  datetime_for_data = datetime.datetime.combine(date_for_data, time_for_data)
  
  #2 minute window
  lower_bound = datetime_for_data - datetime.timedelta(seconds=60)
  upper_bound = datetime_for_data + datetime.timedelta(seconds=60)

  #JSON would work too, but CSV is actually better for D3
  return archive[lower_bound:upper_bound].to_csv()


```

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*